In [1]:
#Aim: Create a movie recommendation System

# Importing Libraries

In [1]:
import pandas as pd

# Loading dataset

In [2]:
MovieMeta = pd.read_csv('../data/IMDB_Dataset.csv',low_memory=False)

In [3]:
#Printing first 5 head values to check data
#pd.set_option('max_columns', None)
MovieMeta.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [24]:
#Data exploration
MovieMeta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

#### The matrix we consider for calculating movie rating without bias here is "weighted average" the formulae is as below:

$ WieghtedAverage = ({\frac {votes_{num}} {votes_{min}+ votes_{num}}}*rating_{avg})+ ({\frac {votes_{min}} {votes_{min}+votes_{num}}}*rating_{overal mean} )$

In [6]:
#Calculating mean vote
MeanVote = MovieMeta['vote_average'].mean()
print(MeanVote)

5.618207215133889


In [7]:
#Finding the minimum value to check the cutoff value
#Note: Here we have taken 91.25%(200 ratings) quantile as the movie which 
#havent been rated at all will lead to minumum number of votes as 0
MinimumVote = MovieMeta['vote_count'].quantile(0.9125)
MinimumVote

200.0

In [8]:
#Filtering all the movies which are atleased rated by 200 people into new dataframe

MoviesList200 = MovieMeta.copy().loc[MovieMeta['vote_count']>= MinimumVote]
MoviesList200.shape

(3982, 24)

This leaves us with just 3982 movies

In [9]:
"""my_values = MovieMeta['vote_count'].round(-1)
my_values.plot"""

"my_values = MovieMeta['vote_count'].round(-1)\nmy_values.plot"

#### Calculating weighted average using the formuala metioned earlier

In [10]:
def WeightedRating(x , MinimumVote=MinimumVote, MeanVote= MeanVote ):
    voters = x['vote_count']
    AvgVote = x['vote_average']
    
    WRating = ((voters/(voters + MinimumVote))*AvgVote)+\
        ((MinimumVote/(voters+ MinimumVote))*MeanVote)
    return WRating

#### Calculating the scores with weighted average

In [11]:
MoviesList200['score'] = MoviesList200.apply(WeightedRating, axis = 1)

In [12]:
MoviesList200 = MoviesList200.sort_values('score', ascending = False)

In [31]:
Top200 = MoviesList200[['title','vote_count','vote_average','score']].round(decimals=2)
#pd.set_option('max_rows', None)
print("**Printing the list of top 200 movies***")
Top200.head(200)


**Printing the list of top 200 movies***


,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.43
834,The Godfather,6024.0,8.5,8.41
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.29
12481,The Dark Knight,12269.0,8.3,8.26
2843,Fight Club,9678.0,8.3,8.25
...,...,...,...,...
6994,Big Fish,2064.0,7.6,7.42
17828,Warrior,1297.0,7.7,7.42
11567,Ratatouille,4510.0,7.5,7.42
1218,The Deer Hunter,943.0,7.8,7.42
